In [1]:
%matplotlib widget

In [2]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json

import os

In [3]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"


In [4]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

# input json name

In [5]:
file_parameters['watershed'] = 'geum_1'

In [6]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [7]:
pd.set_option('display.max_columns', 1000)

In [8]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [9]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [10]:
gain_epochs , rnn_epochs

(2000, 10)

# run file

In [11]:
run_num = range(len(folder))


run_num = [0]




real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)


# Gain Training Flag

In [12]:
__GAIN_TRAINING__ = False

In [13]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #start = time.time()

    #if watershed == '한강_12days_test':
    #    df, times = make_dataframe_temp_12days(folder[idx], file_names[idx], colum_idx[idx], interpolate=interpolation_option[idx])
    #else:
    df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])

    df_all, train_mean, train_std, df = normalize(df)


    

    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/geum/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['현도_2015.xlsx', '현도_2016.xlsx', '현도_2017.xlsx', '현도_2018.xlsx', '현도_2019.xlsx', '현도_2020.xlsx'], ['대청호_2015.xlsx', '대청호_2016.xlsx', '대청호_2017.xlsx', '대청호_2018.xlsx', '대청호_2019.xlsx', '대청호_2020.xlsx'], ['장계_2015.xlsx', '장계_2016.xlsx', '장계_2017.xlsx', '장계_2018.xlsx', '장계_2019.xlsx', '장계_2020.xlsx']]
data/geum/자동/현도_2015.xlsx
data/geum/자동/현도_2016.xlsx
data/geum/자동/현도_2017.xlsx
data/geum/자동/현도_2018.xlsx
data/geum/자동/현도_2019.xlsx
data/geum/자동/현도_2020.xlsx
time range in files :  2015-01-01 00:00  ~  2020-12-31 23:00
data/geum/자동/대청호_2015.xlsx
data/geum/자동/대청호_2016.xlsx
data/geum/자동/대청호_2017.xlsx
data/geum/자동/대청호_2018.xlsx
data/geum/자동/대청호_2019.xlsx
data/geum/자동/대청호_2020.xlsx
time range in files :  2015-01-01 00:00  ~  2020-12-31 23:00
data/geum/자동/장계_2015.xlsx
data/geum/자동/장계_2016.xlsx
data/geum/자동/장계_2017.xlsx
data/geum/자동/장계_2018.xlsx
data/geum/자동/장계_2019.xlsx
data/geum/자동/

In [14]:
print(real_df_all.shape)

(52608, 36)


In [15]:
def dataset_slice(df, train_ratio, val_ratio, test_ratio):
    total_no = df.shape[0]

    train_no = int(total_no * train_ratio)
    #
    val_no = int(total_no * (train_ratio + val_ratio))

    # val_no = int(total_no*val_ratio)
    # train_no =int(total_no * (train_ratio+val_ratio))
    #
    # val_slice = slice(0, val_no)
    # train_slice = slice(val_no, train_no)
    # test_slice = slice(train_no, None)

    train_slice = slice(0, train_no)    #0.8
    val_slice = slice(train_no, val_no)   #0.1
    test_slice = slice(val_no, None)
    # val_slice = slice(train_no, val_no)   #0.1
    # test_slice = slice(val_no, None)
#     train_slice = slice(0, train_no)  # 0.8
#     val_slice = slice(train_no, None)  # 0.1
#     test_slice = slice(val_no, None)

    train = pd.DataFrame(df[train_slice])
    val = pd.DataFrame(df[val_slice])
    test = pd.DataFrame(df[test_slice])

    test_slice2 = slice(total_no - 288, None)
    test2 = pd.DataFrame(df[test_slice2])

    #print('total_no :1111111111111111111111111 ')
    #print('total_no : ', total_no)
    #print('train : ', train.shape)
    #print('val : ', val.shape)
    #print('test : ', test.shape)


    return train, val, test, test2

# slice DF

In [16]:
# fake_df=real_df_all.iloc[:-2000, :]

In [17]:
train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.7, 0.1, 0.2)

In [18]:
val_df.shape, test_df.shape, train_df.shape

((10522, 36), (5261, 36), (36825, 36))

In [19]:
print('-------------------prediction')
print('-------------------prediction')
print('-------------------prediction')

print('real_df_all.type : ', type(real_df_all))
print('train_df.type : ', type(train_df))
print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)


-------------------prediction
-------------------prediction
-------------------prediction
real_df_all.type :  <class 'pandas.core.frame.DataFrame'>
train_df.type :  <class 'pandas.core.frame.DataFrame'>
train_df.shape :  (36825, 36) val_df.shape :  (10522, 36) test_df.shape: (5261, 36)


# target column

In [20]:

label_columns_indices = {name: i for i, name in enumerate(dfff[0])}

print("label_columns_indices:")
print(label_columns_indices)


target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}

rnn_target_column = "chl-a"

print('target columns : ', rnn_target_column)
num_features = dfff[0].shape[1]



target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)


label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target columns :  chl-a
target_col_idx :  7
out_num_features :  1


In [21]:
val_nse = {}
val_pbias = {}


WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df, test_df2=test_df2)

if __RNN_TRAINING__:
    if not os.path.exists('save/' + watershed):
        os.makedirs('save/' + watershed)


idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]

indices = {name: i for i, name in enumerate(idx)}

model_path = "save/" + watershed + "models/" + pa[indices[target_col_idx]]
print("save model path : ", model_path)

val_nse = {}
val_pbias = {}

 # +"gru.ckpt" -- path




save model path :  save/geum/models/chlorophyll-a/


In [22]:
plt.figure()
plt.plot(real_df_all.iloc[:,out_features[0]])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
plt.figure()
plt.plot(val_df.iloc[:,out_features[0]])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 모델 학습

In [24]:
__RNN_TRAINING__ = True

In [25]:
rnn_epochs = 1000
# multi_linear_model = model_multi_linear(
#     window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
#     #training_flag=__RNN_TRAINING__, checkpoint_path="save/"+watershed+"models/multi_linear.ckpt")
#     training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"multi_linear.ckpt")
# elman_model = model_elman(
#     window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
#     training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"elman.ckpt")
gru_model = model_gru(
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt")
# multi_lstm_model = model_multi_lstm(
#     window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
#     training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"multi_lstm.ckpt")
# multi_conv_model = model_multi_conv(
#     window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
#     training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"multi_conv.ckpt")


Epoch 1/1000
10/10 [==============================] - 11s 1s/step - loss: 0.1628 - mean_absolute_error: 0.3632 - nse: -6.1515 - val_loss: 0.1021 - val_mean_absolute_error: 0.2801 - val_nse: -0.9837

Epoch 00001: val_loss improved from inf to 0.10207, saving model to save/geum/models/chlorophyll-a/gru.ckpt
Epoch 2/1000
10/10 [==============================] - 9s 985ms/step - loss: 0.0344 - mean_absolute_error: 0.1533 - nse: -0.5041 - val_loss: 0.0525 - val_mean_absolute_error: 0.1737 - val_nse: -0.0179

Epoch 00002: val_loss improved from 0.10207 to 0.05252, saving model to save/geum/models/chlorophyll-a/gru.ckpt
Epoch 3/1000
10/10 [==============================] - 9s 955ms/step - loss: 0.0264 - mean_absolute_error: 0.1256 - nse: -0.1312 - val_loss: 0.0713 - val_mean_absolute_error: 0.2205 - val_nse: -0.3788

Epoch 00003: val_loss did not improve from 0.05252
Epoch 4/1000
10/10 [==============================] - 9s 947ms/step - loss: 0.0212 - mean_absolute_error: 0.1107 - nse: 0.0637 -

10/10 [==============================] - 9s 956ms/step - loss: 0.0103 - mean_absolute_error: 0.0715 - nse: 0.5461 - val_loss: 0.0421 - val_mean_absolute_error: 0.1655 - val_nse: 0.1871

Epoch 00030: val_loss did not improve from 0.03689
Epoch 31/1000
10/10 [==============================] - 9s 940ms/step - loss: 0.0110 - mean_absolute_error: 0.0751 - nse: 0.5279 - val_loss: 0.0405 - val_mean_absolute_error: 0.1610 - val_nse: 0.2212

Epoch 00031: val_loss did not improve from 0.03689
Epoch 32/1000
10/10 [==============================] - 9s 965ms/step - loss: 0.0110 - mean_absolute_error: 0.0740 - nse: 0.5160 - val_loss: 0.0357 - val_mean_absolute_error: 0.1444 - val_nse: 0.3142

Epoch 00032: val_loss improved from 0.03689 to 0.03568, saving model to save/geum/models/chlorophyll-a/gru.ckpt
Epoch 33/1000
10/10 [==============================] - 9s 947ms/step - loss: 0.0121 - mean_absolute_error: 0.0777 - nse: 0.4621 - val_loss: 0.0453 - val_mean_absolute_error: 0.1770 - val_nse: 0.1256



10/10 [==============================] - 9s 961ms/step - loss: 0.0092 - mean_absolute_error: 0.0690 - nse: 0.5825 - val_loss: 0.0507 - val_mean_absolute_error: 0.1826 - val_nse: 0.0242

Epoch 00062: val_loss did not improve from 0.03568
Epoch 63/1000
10/10 [==============================] - 9s 964ms/step - loss: 0.0096 - mean_absolute_error: 0.0678 - nse: 0.5685 - val_loss: 0.0517 - val_mean_absolute_error: 0.1874 - val_nse: 0.0025

Epoch 00063: val_loss did not improve from 0.03568
Epoch 64/1000
10/10 [==============================] - 9s 962ms/step - loss: 0.0102 - mean_absolute_error: 0.0694 - nse: 0.5932 - val_loss: 0.0599 - val_mean_absolute_error: 0.2067 - val_nse: -0.1530

Epoch 00064: val_loss did not improve from 0.03568
Epoch 65/1000
10/10 [==============================] - 9s 961ms/step - loss: 0.0090 - mean_absolute_error: 0.0692 - nse: 0.5822 - val_loss: 0.0562 - val_mean_absolute_error: 0.1938 - val_nse: -0.0830

Epoch 00065: val_loss did not improve from 0.03568
Epoch 66


Epoch 00094: val_loss did not improve from 0.03568
Epoch 95/1000
10/10 [==============================] - 9s 949ms/step - loss: 0.0078 - mean_absolute_error: 0.0620 - nse: 0.6559 - val_loss: 0.0677 - val_mean_absolute_error: 0.2174 - val_nse: -0.3080

Epoch 00095: val_loss did not improve from 0.03568
Epoch 96/1000
10/10 [==============================] - 9s 943ms/step - loss: 0.0072 - mean_absolute_error: 0.0605 - nse: 0.6727 - val_loss: 0.0676 - val_mean_absolute_error: 0.2163 - val_nse: -0.3067

Epoch 00096: val_loss did not improve from 0.03568
Epoch 97/1000
10/10 [==============================] - 9s 979ms/step - loss: 0.0076 - mean_absolute_error: 0.0618 - nse: 0.6752 - val_loss: 0.0651 - val_mean_absolute_error: 0.2120 - val_nse: -0.2585

Epoch 00097: val_loss did not improve from 0.03568
Epoch 98/1000
10/10 [==============================] - 9s 932ms/step - loss: 0.0075 - mean_absolute_error: 0.0607 - nse: 0.6746 - val_loss: 0.0698 - val_mean_absolute_error: 0.2223 - val_nse: 

10/10 [==============================] - 9s 951ms/step - loss: 0.0067 - mean_absolute_error: 0.0568 - nse: 0.7050 - val_loss: 0.0713 - val_mean_absolute_error: 0.2211 - val_nse: -0.3728

Epoch 00127: val_loss did not improve from 0.03568
Epoch 128/1000
10/10 [==============================] - 9s 944ms/step - loss: 0.0068 - mean_absolute_error: 0.0571 - nse: 0.6844 - val_loss: 0.0726 - val_mean_absolute_error: 0.2218 - val_nse: -0.4040

Epoch 00128: val_loss did not improve from 0.03568
Epoch 129/1000
10/10 [==============================] - 9s 980ms/step - loss: 0.0067 - mean_absolute_error: 0.0575 - nse: 0.7138 - val_loss: 0.0656 - val_mean_absolute_error: 0.2117 - val_nse: -0.2687

Epoch 00129: val_loss did not improve from 0.03568
Epoch 130/1000
10/10 [==============================] - 9s 966ms/step - loss: 0.0068 - mean_absolute_error: 0.0571 - nse: 0.7177 - val_loss: 0.0709 - val_mean_absolute_error: 0.2206 - val_nse: -0.3710

Epoch 00130: val_loss did not improve from 0.03568
Epo

...

10/10 [==============================] - 9s 959ms/step - loss: 0.0031 - mean_absolute_error: 0.0344 - nse: 0.8657 - val_loss: 0.0717 - val_mean_absolute_error: 0.2251 - val_nse: -0.3827

Epoch 00863: val_loss did not improve from 0.03568
Epoch 864/1000
10/10 [==============================] - 9s 947ms/step - loss: 0.0031 - mean_absolute_error: 0.0345 - nse: 0.8601 - val_loss: 0.0717 - val_mean_absolute_error: 0.2263 - val_nse: -0.3843

Epoch 00864: val_loss did not improve from 0.03568
Epoch 865/1000
10/10 [==============================] - 9s 974ms/step - loss: 0.0030 - mean_absolute_error: 0.0340 - nse: 0.8694 - val_loss: 0.0725 - val_mean_absolute_error: 0.2276 - val_nse: -0.4012

Epoch 00865: val_loss did not improve from 0.03568
Epoch 866/1000
10/10 [==============================] - 9s 964ms/step - loss: 0.0030 - mean_absolute_error: 0.0339 - nse: 0.8641 - val_loss: 0.0688 - val_mean_absolute_error: 0.2217 - val_nse: -0.3288

Epoch 00866: val_loss did not improve from 0.03568
Epo

10/10 [==============================] - 9s 955ms/step - loss: 0.0047 - mean_absolute_error: 0.0449 - nse: 0.7890 - val_loss: 0.0689 - val_mean_absolute_error: 0.2245 - val_nse: -0.3311

Epoch 00895: val_loss did not improve from 0.03568
Epoch 896/1000
10/10 [==============================] - 9s 934ms/step - loss: 0.0052 - mean_absolute_error: 0.0497 - nse: 0.7731 - val_loss: 0.0652 - val_mean_absolute_error: 0.2113 - val_nse: -0.2513

Epoch 00896: val_loss did not improve from 0.03568
Epoch 897/1000
10/10 [==============================] - 9s 944ms/step - loss: 0.0042 - mean_absolute_error: 0.0438 - nse: 0.8254 - val_loss: 0.0718 - val_mean_absolute_error: 0.2263 - val_nse: -0.3859

Epoch 00897: val_loss did not improve from 0.03568
Epoch 898/1000
10/10 [==============================] - 9s 942ms/step - loss: 0.0036 - mean_absolute_error: 0.0404 - nse: 0.8319 - val_loss: 0.0711 - val_mean_absolute_error: 0.2218 - val_nse: -0.3742

Epoch 00898: val_loss did not improve from 0.03568
Epo

10/10 [==============================] - 9s 979ms/step - loss: 0.0030 - mean_absolute_error: 0.0343 - nse: 0.8568 - val_loss: 0.0715 - val_mean_absolute_error: 0.2255 - val_nse: -0.3801

Epoch 00927: val_loss did not improve from 0.03568
Epoch 928/1000
10/10 [==============================] - 9s 953ms/step - loss: 0.0032 - mean_absolute_error: 0.0344 - nse: 0.8670 - val_loss: 0.0712 - val_mean_absolute_error: 0.2262 - val_nse: -0.3745

Epoch 00928: val_loss did not improve from 0.03568
Epoch 929/1000
10/10 [==============================] - 9s 959ms/step - loss: 0.0031 - mean_absolute_error: 0.0341 - nse: 0.8712 - val_loss: 0.0705 - val_mean_absolute_error: 0.2250 - val_nse: -0.3629

Epoch 00929: val_loss did not improve from 0.03568
Epoch 930/1000
10/10 [==============================] - 9s 950ms/step - loss: 0.0031 - mean_absolute_error: 0.0343 - nse: 0.8596 - val_loss: 0.0713 - val_mean_absolute_error: 0.2254 - val_nse: -0.3734

Epoch 00930: val_loss did not improve from 0.03568
Epo

10/10 [==============================] - 9s 945ms/step - loss: 0.0033 - mean_absolute_error: 0.0353 - nse: 0.8681 - val_loss: 0.0692 - val_mean_absolute_error: 0.2225 - val_nse: -0.3356

Epoch 00959: val_loss did not improve from 0.03568
Epoch 960/1000
10/10 [==============================] - 9s 959ms/step - loss: 0.0031 - mean_absolute_error: 0.0348 - nse: 0.8682 - val_loss: 0.0720 - val_mean_absolute_error: 0.2264 - val_nse: -0.3963

Epoch 00960: val_loss did not improve from 0.03568
Epoch 961/1000
10/10 [==============================] - 9s 963ms/step - loss: 0.0028 - mean_absolute_error: 0.0345 - nse: 0.8796 - val_loss: 0.0710 - val_mean_absolute_error: 0.2250 - val_nse: -0.3727

Epoch 00961: val_loss did not improve from 0.03568
Epoch 962/1000
10/10 [==============================] - 9s 960ms/step - loss: 0.0030 - mean_absolute_error: 0.0327 - nse: 0.8637 - val_loss: 0.0703 - val_mean_absolute_error: 0.2235 - val_nse: -0.3605

Epoch 00962: val_loss did not improve from 0.03568
Epo

10/10 [==============================] - 9s 952ms/step - loss: 0.0031 - mean_absolute_error: 0.0349 - nse: 0.8695 - val_loss: 0.0687 - val_mean_absolute_error: 0.2223 - val_nse: -0.3295

Epoch 00991: val_loss did not improve from 0.03568
Epoch 992/1000
10/10 [==============================] - 9s 942ms/step - loss: 0.0028 - mean_absolute_error: 0.0339 - nse: 0.8756 - val_loss: 0.0722 - val_mean_absolute_error: 0.2284 - val_nse: -0.3932

Epoch 00992: val_loss did not improve from 0.03568
Epoch 993/1000
10/10 [==============================] - 9s 937ms/step - loss: 0.0030 - mean_absolute_error: 0.0343 - nse: 0.8739 - val_loss: 0.0728 - val_mean_absolute_error: 0.2291 - val_nse: -0.4032

Epoch 00993: val_loss did not improve from 0.03568
Epoch 994/1000
10/10 [==============================] - 9s 968ms/step - loss: 0.0028 - mean_absolute_error: 0.0338 - nse: 0.8773 - val_loss: 0.0715 - val_mean_absolute_error: 0.2269 - val_nse: -0.3824

Epoch 00994: val_loss did not improve from 0.03568
Epo

## core / window.py / 

In [26]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    
    print(df.shape)
    print(plot_col)
    
    width = input_width + label_width
    
    length = df.shape[0]
    length -= width
    
    inputs = []
    labels = []
   
    for i in range(0,length,24):
#         print('i = ', i)
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    
#     print('labels.mean(axis=1)')
#     print(labels.mean(axis=1).shape)
    print(inputs.shape)
    print(labels.shape)

    predictions = model(inputs)
    print(predictions.shape)
    
    predictions = predictions.numpy() * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    print('predictions.shape =', predictions.shape)
#     pred_day = predictions
    pred_day = hour_to_day_mean(predictions)
    print('pred_day.shape =', pred_day.shape)
#     label_day= labels
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
#     inputs_day = inputs_target
    inputs_day = hour_to_day_mean(inputs_target)
    
    #plt.figure(figsize=(10, 800))
    
#     input_index = np.array(range(0,length,24))
#     label_index = input_index + 24*7
    
#     print(input_index)
#     print(label_index)
    
    plt.figure()
    plt.plot(label_day[:, predict_day, :], label='label')
    plt.plot(pred_day[:, predict_day, :], label='pred')
#     plt.plot(input_index, inputs_day[:, 0, :], label='input')
#     plt.plot(label_index, label_day[:, 0, :], label='label')
#     plt.plot(label_index, pred_day[:, 0, :], label='pred')
    plt.legend()
        
        
        
#     o1 = np.mean(labels)
# #     print('labels =', labels)
#     print('o1 =', o1)
#     nse1 = ((labels - predictions)**2).sum()
#     nse2 = ((labels - o1)**2).sum()
#     nse3 = 1 - (nse1/nse2)
    
    
    
#     print('pred_day.shape =',pred_day.shape)
#     print('label_day.shape =',label_day.shape)
    #print('label_index =', label_index)
    
    o1 = np.mean(label_day[:,predict_day,0])
#     o1 = np.mean(label_day)
#     print('labels =', labels)
#     o1 = target_mean[plot_col]
    print('o1 =', o1)
    nse1 = ((pred_day-label_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('nse3 =', nse3)
    #mean = np.mean(label_day[predict_day])
    print('label_day[predict_day].shape =', label_day[predict_day].shape)
#     print(type(pred_day))
#     print(pred_day[:20, 0, 0])
#     print(label_day[:20, 0, 0])
    o = label_day[:, 0, 0]
    s = pred_day[:, 0, 0]
    mean = np.mean(o)
    print('mean =', mean)
    nse = 1. - np.sum((o-s)**2)/np.sum((o-mean)**2)
#     print('nse =', nse)
    
    
#     pbias1 = (label_day - pred_day).sum(axis=0)
#     pbias2 = (label_day).sum(axis=0)
#     pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
#     o1 = np.mean(label_day)
# #     print('labels =', labels)
#     print('o1 =', o1)
#     nse1 = ((label_day - pred_day)**2).sum()
#     nse2 = ((label_day - o1)**2).sum()
#     nse3 = 1 - (nse1/nse2)
#     print('nse3 =', nse3)
    #mean = np.mean(label_day[predict_day])
#     print(type(pred_day))
#     print(pred_day[:20, 0, 0])
#     print(label_day[:20, 0, 0])
#     o = label_day[:, 0, 0]
#     s = pred_day[:, 0, 0]
#     mean = np.mean(o)
#     print('mean =', mean)
#     nse = 1. - np.sum((o-s)**2)/np.sum((o-mean)**2)
#     print('nse =', nse)
    
    
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    
    
    return nse3, np.abs(pbias3), pred_day, labels
    

In [27]:
test_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35
36825,-1.142398,-0.550392,0.664851,-0.638951,-0.299066,-0.354457,-0.284360,-0.572070,9.999968e-01,-0.999997,1.348457,0.426324,-1.027049,-0.406150,1.064717,-0.957662,0.450288,-0.427551,-0.204083,-0.506668,9.999968e-01,-0.999997,1.348457,0.426324,-0.877716,-0.417242,1.363400,1.113961,0.766799,1.792998,0.384619,0.472809,9.999968e-01,-0.999997,1.348457,0.426324
36826,-1.147547,-0.550392,0.655449,-0.625376,-0.291223,-0.355367,-0.334162,-0.597302,7.071046e-01,-1.224741,1.348763,0.425358,-0.986884,-0.284092,1.082556,-0.957662,0.439971,-0.388439,-0.327472,-0.619805,7.071046e-01,-1.224741,1.348763,0.425358,-0.855058,-0.417242,1.383185,1.113961,0.660801,1.536653,0.384619,0.558808,7.071046e-01,-1.224741,1.348763,0.425358
36827,-1.143427,-0.550392,0.665307,-0.629390,-0.241244,-0.398366,-0.433022,-0.584433,3.660243e-01,-1.366021,1.349067,0.424391,-0.970406,-0.284092,1.085798,-0.957662,0.467955,-0.530845,-0.321525,-0.630989,3.660243e-01,-1.366021,1.349067,0.424391,-0.858148,-0.417242,1.404262,1.113961,0.763866,1.779570,0.316234,1.754279,3.660243e-01,-1.366021,1.349067,0.424391
36828,-1.148577,-0.550392,0.689824,-0.619474,-0.313782,-0.404791,-0.395856,-0.579203,2.767791e-12,-1.414209,1.349372,0.423424,-0.964226,-0.350667,1.072177,-0.957662,0.526837,-0.623040,-0.550465,-0.648089,2.767791e-12,-1.414209,1.349372,0.423424,-0.859178,-0.417242,1.414964,1.108206,0.743250,1.562477,0.235957,2.005433,2.767791e-12,-1.414209,1.349372,0.423424
36829,-1.155786,-0.550392,0.680745,-0.585832,-0.290233,-0.302652,-0.470187,-0.529604,-3.660243e-01,-1.366021,1.349675,0.422457,-0.975556,-0.284092,1.083852,-0.957662,0.422305,-0.568189,-0.290306,-0.481221,-3.660243e-01,-1.366021,1.349675,0.422457,-0.856088,-0.328471,1.436695,1.104665,0.656384,1.429420,0.371982,2.146205,-3.660243e-01,-1.366021,1.349675,0.422457
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42081,0.302539,-1.216144,-2.051832,-1.170136,0.671118,-0.501520,0.979266,-0.615104,9.999968e-01,-0.999997,-1.342900,0.443509,0.690808,-0.284092,-0.731595,-0.260039,-1.211835,-0.477313,-0.196506,0.239579,9.999968e-01,-0.999997,-1.342900,0.443509,0.636223,-0.273000,-0.240600,-0.420280,2.857332,-0.433083,-0.507353,0.979586,9.999968e-01,-0.999997,-1.342900,0.443509
42082,0.306658,-1.216144,-1.998645,-1.170136,0.640202,-0.489843,1.146511,-0.619237,7.071046e-01,-1.224741,-1.342581,0.444471,0.700077,-0.284092,-0.691056,-0.235693,-1.256001,-0.442970,-0.157132,0.267619,7.071046e-01,-1.224741,-1.342581,0.444471,0.636223,-0.150941,-0.188389,-0.426477,2.654168,-0.262385,-0.507353,1.020270,7.071046e-01,-1.224741,-1.342581,0.444471
42083,0.309748,-1.282719,-2.068372,-1.154201,0.641668,-0.572706,1.313755,-0.643138,3.660243e-01,-1.366021,-1.342262,0.445433,0.726853,-0.051078,-0.490637,-0.311386,-1.226551,-0.449915,-0.150201,0.312598,3.660243e-01,-1.366021,-1.342262,0.445433,0.647553,0.070980,-0.083638,-0.426477,2.710100,-0.436585,-0.507353,1.286821,3.660243e-01,-1.366021,-1.342262,0.445433
42084,0.310778,-1.271628,-2.057020,-1.144463,0.672584,-0.549228,1.127928,-0.657637,9.123998e-12,-1.414209,-1.341942,0.446395,0.778348,-0.017791,-0.482208,-0.320240,-1.141169,-0.452696,-0.263741,-0.090754,9.123998e-12,-1.414209,-1.341942,0.446395,0.667120,0.159738,-0.038884,-0.426477,2.892664,-0.406244,-0.451605,1.630283,9.123998e-12,-1.414209,-1.341942,0.446395


In [28]:
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(model=gru_model,
                                                      df=val_df,
                                                      plot_col=out_features[0], 
                                                      target_std=target_std, 
                                                      target_mean=target_mean,
                                                      predict_day = 4)

print(val_nse['GRU'])
print(val_pbias['GRU'])

(10522, 36)
7
(427, 168, 36)
(427, 120, 1)
(427, 120, 1)
predictions.shape = (427, 120, 1)
pred_day.shape = (427, 5, 1)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 = 3.7838168
nse3 = [0.32085258]
label_day[predict_day].shape = (5, 1)
mean = 3.7613049
[0.32085258]
[11.711661]


In [29]:
val_nse['LSTM'], val_pbias['LSTM'], pred, label = compa(model=multi_lstm_model,
                                                      df=val_df,
                                                      plot_col=out_features[0], 
                                                      target_std=target_std, 
                                                      target_mean=target_mean,
                                                      predict_day = 4)

print(val_nse['LSTM'])
print(val_pbias['LSTM'])

NameError: name 'multi_lstm_model' is not defined

In [ ]:
val_nse['CNN'], val_pbias['CNN'], pred, label = compa(model=multi_conv_model,
                                                      df=val_df,
                                                      plot_col=out_features[0], 
                                                      target_std=target_std, 
                                                      target_mean=target_mean,
                                                      predict_day = 4)

print(val_nse['CNN'])
print(val_pbias['CNN'])

In [ ]:
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=train_df,
    plot_col=out_features[0], target_std=target_std, target_mean=target_mean, predict_day = 4)

print()
print()

print(val_nse['GRU'])
print(val_pbias['GRU'])

In [ ]:
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=real_df_all,
    plot_col=out_features[0], target_std=target_std, target_mean=target_mean, predict_day = 4)

print()
print()

print(val_nse['GRU'])
print(val_pbias['GRU'])